In [1]:
!pip install netCDF4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.5 MB/s eta 0:00:00


In [2]:
def ref():
    !rm *.nc
    !rm -r season_avg_yr_data_files

    # make directory for each season

    !mkdir season_avg_yr_data_files

    !mkdir season_avg_yr_data_files/DJF_avg


    !mkdir season_avg_yr_data_files/MAM_avg


    !mkdir season_avg_yr_data_files/JJA_avg


    !mkdir season_avg_yr_data_files/SON_avg

In [11]:
ref()

In [ ]:
import shutil

# Create a zip archive of the 'avg_season' folder
shutil.make_archive('year_avg_season_archive', 'zip', '.', 'season_avg_yr_data_files')



'/content/year_avg_season_archive.zip'

In [4]:
!rm *.nc


rm: cannot remove '*.nc': No such file or directory


In [4]:
# !cp /content/file_path/source_file.txt /content/new_directory/destination_file.txt


In [12]:
# prompt: copy files from BTP folder in drive to main folder

!cp /content/drive/My\ Drive/BTP/*.nc /content/


In [1]:
# prompt: connect with onedrive and mount it

from google.colab import drive
drive.mount('/content/drive')


MessageError: ignored

In [5]:
import netCDF4 as nc
import numpy as np
import requests
import os
import glob

def calculate_and_save_yr_seasonal_averages(urls):
  for url_group in urls:
    n = len(url_group)
    for i in range(n):
      url = url_group[i]
      response = requests.get(url)
      filename = url.split('/')[-1]
      with open(filename, 'wb') as f:
        f.write(response.content)
        print(f"{filename} download done")


  files = glob.glob("*.nc")
  dp=nc.Dataset(files[0])
  latitude=dp.variables['lat'][:]
  longitude=dp.variables['lon'][:]
  print(files)
  seasons = {
        'DJF': (12, 1, 2),
        'MAM': (3, 4, 5),
        'JJA': (6, 7, 8),
        'SON': (9, 10, 11)
    }
  yearly_seasonal_avgs = {season: [] for season in seasons}
  for file in files:
    ds = nc.Dataset(file)
    season_data = ds.variables['zmla']
    lat=ds.variables['lat'][:]
    lon=ds.variables['lon'][:]
    length=len(season_data)
    print(length)

    # print(season_data)
    # Get the time variable
    time_var = ds['time']

    # Convert numeric time values to datetime objects
    time_values = time_var[:]
    time_units = time_var.units
    calendar = time_var.calendar if hasattr(
        time_var, 'calendar') else 'standard'

    dates = nc.num2date(time_values, units=time_units, calendar=calendar)

    # Define the start and end months for each season (assuming a calendar year)


    # Loop through each year and each season, compute the average, and store it
    for year in range(dates[0].year, dates[-1].year + 1):
        for season, (start_month, mid_month, end_month) in seasons.items():
            # Find indices of dates within the current season and year
            season_indices = [
                i for i, date in enumerate(dates) if (date.month == start_month or date.month == mid_month or date.month == end_month) and date.year == year
            ]

            # Extract data for the current season and year
            season_data = ds.variables['zmla'][season_indices, :, :]

            # Calculate the seasonal average for the current season and year
            seasonal_avg = np.mean(season_data, axis=0)

            # Store the seasonal average in the dictionary
            yearly_seasonal_avgs[season].append(seasonal_avg)

    ds.close()
  print(len(yearly_seasonal_avgs['DJF']))
  for season in seasons:
    # create the new file within season name folder having that season's average
    season_avg_file = f'season_avg_yr_data_files/{season}_avg/{filename}'

    ncfile = nc.Dataset(season_avg_file, mode='w', format='NETCDF4')

    dim1 = ncfile.createDimension('dim1', len(yearly_seasonal_avgs[season]))
    # dim2 = ncfile.createDimension('dim2', yearly_seasonal_avgs[season].shape[1])
    # dim5 = ncfile.createDimension('dim5', yearly_seasonal_avgs[season].shape[2])

    dim3 = ncfile.createDimension('dim3', latitude.shape[0])
    dim4 = ncfile.createDimension('dim4', longitude.shape[0])

  # create the variable

  # latshape=np.shape(lat)

    var = ncfile.createVariable('syraverage', 'f4', ('dim1', 'dim3','dim4'))
    varlat = ncfile.createVariable('lat', 'f4',dim3)
    varlot = ncfile.createVariable('lon', 'f4',dim4)
    # latitude = ncfile.createVariable('lat', 'f4', ('dim1', 'dim2', 'dim3'))

    # write the data to the variable
    var[:] = yearly_seasonal_avgs[season][:]
    varlat[:]=lat[:]
    varlot[:]=lon[:]

  #     Close the new netCDF file
    ncfile.close()

    !rm * .nc
    print("avg1 done")




In [ ]:
# CESM2 ! 1
urls1 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190326/zmla_Eday_CESM2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]

urls2 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls3 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

urls4 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_20100101-20150101.nc"
    ]

urls5 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_20100101-20150101.nc"
    ]
urls6 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_20100101-20150101.nc"
    ]
urls7 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_20100101-20150101.nc"
    ]
urls8 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_20100101-20150101.nc"
    ]
urls9 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_20100101-20150101.nc"
    ]
urls10 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_20100101-20150101.nc"
    ]


urls_1 = [urls1,urls2,urls3,urls4,urls5,urls6,urls7,urls7,urls8,urls9,urls10]
calculate_and_save_yr_seasonal_averages(urls_1)


In [6]:
# CESM2 FV2 ! 2

urls21 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]
urls22 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]
urls23 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]
urls24 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20100101-20150103.nc"
    ]
urls2 = [urls21,urls22,urls23,urls24]

calculate_and_save_yr_seasonal_averages(urls2)

zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc download done
zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc download done
zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc downlo

OSError: ignored

In [ ]:
# CESM2 WACCM ! 3
urls31 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20100101-20150116.nc"
    ]
urls32 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls33 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

urls3 = [urls31,urls32,urls33]

calculate_and_save_yr_seasonal_averages(urls3)

zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20000101-20091231.nc download done
zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20100101-20150116.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20000101-20091231.nc download done
zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn

OSError: ignored

In [ ]:
# MIROC-ES2L ! 11

urls111 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]


urls112 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls119 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]
urls11 = [urls111,urls112,urls119]

calculate_and_save_yr_seasonal_averages(urls11)

zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc download done
zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-1992